In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2 

In [2]:
import logging
import os
import time

import numpy as np
import matplotlib.pyplot as plt
import foolbox
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim

from lib.dataset_utils import *
from lib.mnist_model import *
from lib.adv_model import *
from lib.dknn_attack_v2 import DKNNAttackV2
from lib.cwl2_attack import CWL2Attack
from lib.dknn import DKNNL2
from lib.knn_defense import *
from lib.utils import *
from lib.lip_model import *

Loading faiss with AVX2 support.


In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
# Set all random seeds
seed = 2020
np.random.seed(seed)
torch.manual_seed(seed)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
(x_train, y_train), (x_valid, y_valid), (x_test, y_test) = load_mnist_all(
    '/data', val_size=0.1, shuffle=True, seed=seed)

In [26]:
# model_name = 'mnist_basic.h5'
# net = BasicModel()

model_name = 'adv_mnist_exp6.h5'
basic_net = BasicModel()
config = {'epsilon': 0.3,
          'num_steps': 40,
          'step_size': 0.01,
          'random_start': True,
          'loss_func': 'xent'}
net = PGDL2Model(basic_net, config)

In [28]:
# Set up model directory
# save_dir = os.path.join(os.getcwd(), 'saved_models/mnist/')
save_dir = os.path.join(os.getcwd(), 'saved_models/')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)

net = net.to(device)
# if device == 'cuda':
#     net = torch.nn.DataParallel(net)
#     cudnn.benchmark = True
net.load_state_dict(torch.load(model_path))
# net = net.module
net = net.basic_net
net.eval()

BasicModel(
  (conv1): Conv2d(1, 64, kernel_size=(8, 8), stride=(2, 2), padding=(3, 3))
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(64, 128, kernel_size=(6, 6), stride=(2, 2), padding=(3, 3))
  (relu2): ReLU(inplace=True)
  (conv3): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1))
  (relu3): ReLU(inplace=True)
  (fc): Linear(in_features=2048, out_features=10, bias=True)
)

In [29]:
layers = ['conv1', 'conv2', 'conv3']
knn = CVPR_Defense(net, x_train, y_train, layers, 
                   k=50, num_classes=10)

In [30]:
with torch.no_grad():
    y_pred = knn.get_output(x_test)
    ind = np.where(y_pred.argmax(1) == y_test.numpy())[0]
    print((y_pred.argmax(1) == y_test.numpy()).sum() / y_test.size(0))

0.968


In [35]:
def attack_batch(attack, x, y, init_mode, init_mode_k, batch_size):
    x_adv = torch.zeros_like(x)
    total_num = x.size(0)
    num_batches = total_num // batch_size
    for i in range(num_batches):
        begin = i * batch_size
        end = (i + 1) * batch_size
        x_adv[begin:end] = attack(
            x[begin:end], y[begin:end], 2, m=200,
            init_mode=init_mode, init_mode_k=init_mode_k,
            binary_search_steps=10, max_iterations=1000, learning_rate=1e-1,
            initial_const=1e-2, max_linf=None, random_start=True,
            thres_steps=200, check_adv_steps=200, verbose=False)
    return x_adv

num = 100

def full_eval(knn):
    with torch.no_grad():
        y_pred = knn.get_output(x_test)
        ind = np.where(y_pred.argmax(1) == y_test.numpy())[0]
    print((y_pred.argmax(1) == y_test.numpy()).sum() / y_test.size(0))
    
    dist_all = np.zeros(num) + 1e9
    attack = CVPR_Attack(knn)
    
    x_adv = attack_batch(
        attack, x_test[ind][:num].cuda(), y_test[ind][:num], 1, 1, 100)
    with torch.no_grad():
        y_pred = knn.get_output(x_adv)
        ind_adv = y_pred.argmax(1) != y_test[ind][:num].numpy()
        dist = (x_adv.cpu() - x_test[ind][:num]).view(
            num, -1).norm(2, 1).numpy()
    for i in range(num):
        if ind_adv[i] and (dist[i] < dist_all[i]):
            dist_all[i] = dist[i]
            
    for k in range(1, 6):
        x_adv = attack_batch(
            attack, x_test[ind][:num].cuda(), y_test[ind][:num], 2, k, 100)
        with torch.no_grad():
            y_pred = knn.get_output(x_adv)
            ind_adv = y_pred.argmax(1) != y_test[ind][:num].numpy()
            dist = (x_adv.cpu() - x_test[ind][:num]).view(
                num, -1).norm(2, 1).numpy()
        for i in range(num):
            if ind_adv[i] and (dist[i] < dist_all[i]):
                dist_all[i] = dist[i]
                
    adv_acc = (dist_all == 1e9).mean()
    print('adv accuracy: %.4f, mean dist: %.4f' % (
        adv_acc, dist_all[dist_all < 1e9].mean()))
    return dist_all

In [36]:
start = time.time()
dist = full_eval(knn)
print(time.time() - start)

0.968
adv accuracy: 0.0000, mean dist: 2.3979
654.7994983196259


In [26]:
attack = DKNNAttackV2(dknn)
num = 100
x_adv = attack_batch(
    attack, x_test[ind][:num].cuda(), y_test[ind][:num], 1, 1, 100)

In [34]:
with torch.no_grad():
    y_pred = knn.get_output(x_adv)
    ind_adv = np.where(y_pred.argmax(1) != y_test[ind][:num].numpy())[0]
    adv_acc = (y_pred.argmax(1) == y_test[ind][:num].numpy()).sum() \
        / y_pred.shape[0]
    dist = (x_adv.cpu() - x_test[ind][:num]).view(
        num, -1).norm(2, 1)[ind_adv].mean()
print('adv accuracy: %.4f, mean dist: %.4f' % (adv_acc, dist.item()))

adv accuracy: 0.1600, mean dist: 3.4508


---

In [33]:
attack = CVPR_Attack(knn)
num = 100
x_adv = attack(
    x_test[ind][:num].cuda(), y_test[ind][:num], 2, m=100,
    init_mode=1, init_mode_k=1,
    binary_search_steps=10, max_iterations=1000, learning_rate=1e-1,
    initial_const=1e-2, max_linf=None, random_start=True,
    thres_steps=1000, check_adv_steps=1000, verbose=True)

    step: 0; loss: 1.563; dist: 0.033
    step: 100; loss: 1.497; dist: 0.279
    step: 200; loss: 1.497; dist: 0.287
    step: 300; loss: 1.497; dist: 0.291
    step: 400; loss: 1.498; dist: 0.291
    step: 500; loss: 1.498; dist: 0.293
    step: 600; loss: 1.498; dist: 0.294
    step: 700; loss: 1.497; dist: 0.294
    step: 800; loss: 1.497; dist: 0.293
    step: 900; loss: 1.497; dist: 0.295
binary step: 0; num successful adv: 2/100
binary step: 0; num successful adv so far: 2/100
    step: 0; loss: 15.615; dist: 0.032
    step: 100; loss: 8.965; dist: 1.732
    step: 200; loss: 8.926; dist: 1.739
    step: 300; loss: 8.907; dist: 1.745
    step: 400; loss: 8.908; dist: 1.742
    step: 500; loss: 8.907; dist: 1.743
    step: 600; loss: 8.903; dist: 1.754
    step: 700; loss: 8.906; dist: 1.740
    step: 800; loss: 8.903; dist: 1.739
    step: 900; loss: 8.893; dist: 1.730
binary step: 1; num successful adv: 5/100
binary step: 1; num successful adv so far: 7/100
    step: 0; loss: 15

In [ ]:
2.6985